In [1]:
"""
FYP project imaging
"""

import os
from os.path import exists
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm as tqdm


# Import our own file that has the feature extraction functions
#from extract_features import extract_features
from extract_features import extract_features

In [2]:
#Where is the raw data
file_data = '.' + os.sep + 'data' + os.sep +'metadata.csv'
path_image = '.' + os.sep + 'data' + os.sep + 'images' + os.sep + 'imgs_part_1'
path_mask = '.' + os.sep + 'data' + os.sep + 'images' + os.sep + 'masks_part_1'

# Where we will store the features
file_features = 'features/features.csv'

In [3]:
#Read meta-data into a Pandas dataframe
df = pd.read_csv(file_data)
df.head()

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,...,NaN,NEV,False,False,False,False,False,False,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,5.0,BCC,True,True,False,True,True,True,PAT_46_881_939.png,True
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1545_1867_547.png,False
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,...,5.0,BCC,True,True,False,False,True,True,PAT_684_1302_588.png,True


In [4]:
# Extract image IDs and labels from the data. 
label = np.array(df['diagnostic'])
mask_id = list(os.listdir(path_mask))


# Extract image IDs and labels from the data. 
label = np.array(df['diagnostic'])
mask_id = list(os.listdir(path_mask))

our_list = list(os.listdir(path_image))
filtered_features = df[df["img_id"].isin(our_list)]
image_id = list(filtered_features['img_id'])
filtered_features

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
976,PAT_330,1440,False,False,ITALY,ITALY,72,False,MALE,False,...,4.0,SCC,True,UNK,False,UNK,False,True,PAT_330_1440_532.png,True
983,PAT_373,2598,False,False,BRAZIL,GERMANY,80,False,MALE,False,...,5.0,SEK,False,True,False,False,False,True,PAT_373_2598_514.png,False
984,PAT_1221,766,NaN,NaN,NaN,NaN,54,NaN,NaN,NaN,...,NaN,SEK,True,True,False,False,False,True,PAT_1221_766_430.png,False
988,PAT_150,1799,True,True,POMERANIA,POMERANIA,50,True,FEMALE,True,...,6.0,SCC,True,UNK,True,UNK,False,True,PAT_150_1799_644.png,True
989,PAT_192,296,False,False,ITALY,ITALY,74,True,MALE,False,...,10.0,SCC,True,True,False,True,False,False,PAT_192_296_824.png,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185,PAT_901,1716,False,True,GERMANY,GERMANY,60,True,MALE,False,...,10.0,BCC,False,True,False,True,False,True,PAT_901_1716_497.png,True
1188,PAT_467,904,False,False,POMERANIA,POMERANIA,46,True,FEMALE,False,...,5.0,BCC,True,True,False,False,False,True,PAT_467_904_535.png,True
1342,PAT_270,417,False,False,POMERANIA,POMERANIA,90,False,FEMALE,False,...,7.0,MEL,False,True,False,True,False,False,PAT_270_417_257.png,True
1348,PAT_801,1518,False,False,ITALY,ITALY,43,False,FEMALE,False,...,8.0,MEL,False,True,False,False,False,False,PAT_801_1518_574.png,True


In [5]:
# Use this for the file names, very hacky, might need a rewrite

foo = [x[:-4] for x in image_id]
#bar = [x[:-9] for x in mask_id]
#foo

In [6]:
#Make array to store features
feature_names = ['file_name', 'asymmetry', 'color', 'blue-white_veil']
num_features = len(feature_names)
num_images = len(filtered_features)
features = np.zeros([num_images,num_features], dtype=np.float16)  

#Loop through all images (now just 10 for demonstration)
filename = []
asym=[]
col=[]
for i in tqdm.tqdm(range(10)):
    
    # Define filenames related to this image
    
    file_image = path_image + os.sep + foo[i] + '.png'
    mask_image = path_mask + os.sep + foo[i] + '_mask.png'
        
    if file_image:    
        # Read the image
        x = extract_features(file_image, mask_image)
        
        filename.append(x[0])
        asym.append(x[1])
        col.append(x[2])


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:20<00:00,  2.06s/it]


In [8]:
#Save the image_id used + features to a file   
berg = dict(zip(feature_names, [filename, asym, col, 0]))

df_features = pd.DataFrame.from_dict(berg)     
df_features.to_csv(file_features, index=False)

df_features

,file_name,asymmetry,color,blue-white_veil
0,.\data\images\imgs_part_1\PAT_330_1440_532.png,0.336,1,0
1,.\data\images\imgs_part_1\PAT_373_2598_514.png,0.192,3,0
2,.\data\images\imgs_part_1\PAT_1221_766_430.png,0.259,1,0
3,.\data\images\imgs_part_1\PAT_150_1799_644.png,0.519,2,0
4,.\data\images\imgs_part_1\PAT_192_296_824.png,0.359,1,0
5,.\data\images\imgs_part_1\PAT_1555_2396_71.png,0.127,2,0
6,.\data\images\imgs_part_1\PAT_953_1806_845.png,0.150,2,0
7,.\data\images\imgs_part_1\PAT_164_255_119.png,0.477,1,0
8,.\data\images\imgs_part_1\PAT_490_933_17.png,0.855,2,0
9,.\data\images\imgs_part_1\PAT_1408_1404_929.png,0.117,2,0
